## A financial model to calculate value of "The Coca Cola Company" using the discounted cash flow method 

#### Importing the required libraries

In [1]:
import numpy as np
import pandas as pd

#### Creating a Series called 'sales' to hold the sales data. Index set to years

In [2]:
# defining the start year, years for projections and end year

start_year      = 2019

years_projected = 10

end_year        = start_year + 10

# generating years columns using list comprehension

year            = [str(year)+'E' for year in range(start_year,end_year+1)]

# distinguishing start year as actual data

year[0]         = str(start_year) + 'A' 

In [3]:
# creating the first Series in the columnar data table

sales   = pd.Series(index = year)

In [4]:
# input variable, actual sales data from 2019 Annual Report ($ millions)

sales['2019A'] = 37266 

#### Generating the sales Series 

In [5]:
# sales projections. Future years show NaN as projections are yet to be made. 

print(sales)

2019A    37266.0
2020E        NaN
2021E        NaN
2022E        NaN
2023E        NaN
2024E        NaN
2025E        NaN
2026E        NaN
2027E        NaN
2028E        NaN
2029E        NaN
dtype: float64


#### Assuming a constant growth rate, generating sales projection data for next 10 years using range function

In [6]:
# Input Variable

growth_rate = 0.03

In [7]:
# using loop to generate sales projections

for year in range(1,11):
    
    sales[year] = sales[year - 1] * (1 + growth_rate)

In [8]:
# a look at the sales Series data with projections

print(sales)

2019A    37266.000000
2020E    38383.980000
2021E    39535.499400
2022E    40721.564382
2023E    41943.211313
2024E    43201.507653
2025E    44497.552882
2026E    45832.479469
2027E    47207.453853
2028E    48623.677469
2029E    50082.387793
dtype: float64


#### NOPAT: Arriving at Net Operating Profit after Taxes

In [9]:
# input data, based on actual figures from 2019 Annual report

ebitda_margin    = 0.30                                  

depr_percent     = 0.03                                  

tax_rate         = 0.17                                 

In [10]:
# projecting major financial statement items

ebitda           = sales * ebitda_margin

depreciation     = sales * depr_percent

ebit             = ebitda - depreciation

tax_payment      = -ebit * tax_rate

tax_payment      = tax_payment.apply(lambda x: min(x,0))

In [11]:
# deriving NOPAT (Net Operating Profit after Tax)

def nopat(e, t):
    
    n = e + t
    
    return n

# calling and assigning the nopat function return value to a variable for easy access

no_pat = nopat(ebit, tax_payment)


#### Generating the nopat Series data

In [12]:
# calling function

print(no_pat)

2019A     8351.310600
2020E     8601.849918
2021E     8859.905416
2022E     9125.702578
2023E     9399.473655
2024E     9681.457865
2025E     9971.901601
2026E    10271.058649
2027E    10579.190408
2028E    10896.566121
2029E    11223.463104
dtype: float64


#### Assumption regarding net working capital (nwc) and capex - required to arrive at free cash flow

In [13]:
# input variable, actual data based on 2019 annual report.

# nwc_percent = (current asset less current liabilities) / Sales

# Actual working capital is negative for the company. Yet, we take a small postive figure here. 

nwc_percent     = 0.01    

In [14]:
# deriving key data for net working capital, change in working capital and capex

nwc             = sales * nwc_percent

change_in_nwc   = nwc.shift(1) - nwc

# assuming capital expenditure equals yearly depreciation

capex_percent   = depr_percent

In [15]:
# writing a function to calculate capex

def capex():
   
    return -(sales * capex_percent)

capital_expenditure = capex()

#### Arriving at free cash flow 

In [16]:
# defining a function to calculate free cash flow

# invoking a function inside a function

def free_cash_flow(a = depreciation, b = change_in_nwc, c = no_pat, d = capital_expenditure):
    
    return a + b + c + d

#### Generating the free cash flow data Series for projected years

In [17]:
# assigning the function return value to a variable for easy access

fcf = free_cash_flow()

# invoking the free cash flow function to generate the required series

print(fcf)

2019A             NaN
2020E     8590.670118
2021E     8848.390222
2022E     9113.841928
2023E     9387.257186
2024E     9668.874902
2025E     9958.941149
2026E    10257.709383
2027E    10565.440665
2028E    10882.403885
2029E    11208.876001
dtype: float64


### Calculating beta in order to arrive at cost of equity using CAPM 

#### Importing the daily return stock (Coca Cola) and Dow market return data for last one year. Source Yahoo! finance

In [18]:
df = pd.read_excel("beta.xlsx")

In [19]:
# a quick look at the sample of returns

df.head(4) 

,Coke,Dow
0,-0.002201,0.002755
1,0.006801,0.008458
2,0.006208,0.001673
3,-0.008225,0.001364


#### To calculate beta of the stock we need to cast the data into an numpy array

In [20]:
# generating a numpy array using the dataframe value property

np_array = df.values

# assigning the second column to variable 'm'
# the 1 represents second column in the array (market return)

m = np_array[:,1]       

# assigning the first column to variable 'n'
# the 0 represents first column in the array (stock return)

s = np_array[:,0]       

#### Calculating beta using covariance formula

In [21]:
# generating the covariance matrix for the two arrays - m and s

covariance = np.cov(m,s)

# calculating beta from the covariance matrix

beta = covariance[1,0]/covariance[0,0]

# output beta

print(beta)

0.7482084608985896


#### Calculating cost of equity using CAPM formula

In [22]:
# input variables, assumed long term return of the stock market

market_return     = 0.06                                   

# long term govt. bond rate assumed

risk_free_rate    = 0.03                                   

In [23]:
# Using CAPM formula, cost of equity = risk free return + risk premium x beta

# risk premium = market return - risk free return

def coe(m, r, b):
    
    premium           = m - r

    cost_of_equity    = r + premium * b
    
    return cost_of_equity
    

In [24]:
# assigning coe function's return value to a variable

cost_of_equity = coe(market_return, risk_free_rate, beta)

# output by calling function

print(cost_of_equity)

0.05244625382695769


#### Calculating the present value discount factors for projection years using list comprehension

In [25]:
# using list comprehension

discount_factors = [(1/(1+cost_of_equity))**i for i in range(1,11)]

In [26]:
# generating the discount factors for first four projection years (sample)

discount_factors[:4]   

[0.9501672853732436,
 0.9028178701935589,
 0.8578280049082673,
 0.8150801067408338]

#### Calculating the terminal value and discounted cash flows

In [27]:
# input variable, terminal year growth assumed

terminal_growth = 0.02

In [28]:
# calculating terminal value 

terminal_value = ((fcf[-1] * (1 + terminal_growth))/(cost_of_equity - terminal_growth))

# calculating dcf value as the sum of discounted cash flows of projected years and terminal value discounted back to start year 

dcf_value      = (sum(fcf[1:] * discount_factors) + terminal_value * discount_factors[-1])

In [29]:
# output, the present value of future cash flows accruing to equity (in $ millions)

print(dcf_value)

285569.20530573797


#### Bringing it all together - completing full dataframe

In [30]:
output = pd.DataFrame([sales, ebit, tax_payment, no_pat, 
                       depreciation, capital_expenditure, change_in_nwc,
                       fcf],
                    index=["Sales", "EBIT", "Tax Expense","NOPAT", 
                            "D&A Expense", "Capital Expenditures", "Increase in NWC",
                            "Free Cash Flow"]).round(2)

In [31]:
# gnenerating the dataframe

print(output)

                         2019A     2020E     2021E     2022E     2023E  \
Sales                 37266.00  38383.98  39535.50  40721.56  41943.21   
EBIT                  10061.82  10363.67  10674.58  10994.82  11324.67   
Tax Expense           -1710.51  -1761.82  -1814.68  -1869.12  -1925.19   
NOPAT                  8351.31   8601.85   8859.91   9125.70   9399.47   
D&A Expense            1117.98   1151.52   1186.06   1221.65   1258.30   
Capital Expenditures  -1117.98  -1151.52  -1186.06  -1221.65  -1258.30   
Increase in NWC            NaN    -11.18    -11.52    -11.86    -12.22   
Free Cash Flow             NaN   8590.67   8848.39   9113.84   9387.26   

                         2024E     2025E     2026E     2027E     2028E  \
Sales                 43201.51  44497.55  45832.48  47207.45  48623.68   
EBIT                  11664.41  12014.34  12374.77  12746.01  13128.39   
Tax Expense           -1982.95  -2042.44  -2103.71  -2166.82  -2231.83   
NOPAT                  9681.46   9971

In [32]:
output.to_excel('Python_DCF_Model.xlsx') # data exported to an Excel file